In [4]:
# importing all modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import numpy as np
from osgeo import gdal
import osr
from IPython.display import display, clear_output
%matplotlib inline



# function to read in a data cube from a geo tiff file
def geotiff_to_datacube(fname):
    
    ds = gdal.Open(fname)
    
    geotransform = ds.GetGeoTransform()
    
    proj = osr.SpatialReference(wkt=ds.GetProjection())
    epsg = int(proj.GetAttrValue('AUTHORITY',1))
    
    xy_shape = np.array(ds.GetRasterBand(1).ReadAsArray()).shape
    
    # get number of bands in raster file
    n_bands = ds.RasterCount
    
    # initialize a data cube
    xyz_shape = xy_shape + (n_bands,)
    data_cube = np.ndarray(xyz_shape)
    
    # fill it with bands
    for i in range(1,n_bands+1):
        data_cube[:,:,i-1] =  np.array(ds.GetRasterBand(i).ReadAsArray())
    
    return data_cube, geotransform, epsg
    # end of read in datacube function


# global variables
rois = ['roi1','roi2','roi3']

folder_as = f'data/arealstatistik/'
folder_coefficients = f'data/coefficients/'
folder_composites = f'data/composites/'
folder_output = f'data/classification_data/'

spectral_bands = ['blue','green','red','nir','swir1','swir2']

yearAS = 2004 # 2004 or 2013
        

In [5]:
# label coefficients

# creating name of features
coefficients = ['c','a1','b1']

kernelv = ['top','mid','lower']
kernelh = ['left','center','right']
positions = [f'{v}{h}' for v in kernelv for h in kernelh]

feature_names = [f'{pos}_{band}_{coef}' for pos in positions for band in spectral_bands for coef in coefficients]

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []


for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_coefficients, geotransform, epsg = geotiff_to_datacube(f'{folder_coefficients}coefficients_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    

        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
    
        kernel = dc_coefficients[irow-1:irow+2,icol-1:icol+2,:]
        features[i,:] = kernel.flatten()
        
    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    

data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}coefficients_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()
        

roi1
roi2
roi3


,roi,X,Y,land_cover,topleft_blue_c,topleft_blue_a1,topleft_blue_b1,topleft_green_c,topleft_green_a1,topleft_green_b1,...,lowerright_red_b1,lowerright_nir_c,lowerright_nir_a1,lowerright_nir_b1,lowerright_swir1_c,lowerright_swir1_a1,lowerright_swir1_b1,lowerright_swir2_c,lowerright_swir2_a1,lowerright_swir2_b1
0,roi1,667600,252600,2,325.552948,55.630993,32.489609,515.907349,-14.854885,89.555176,...,64.624893,2389.831055,-1125.281372,-215.238480,1259.682617,-328.539520,-0.342632,640.255493,-79.420906,60.188084
1,roi1,667700,252600,3,284.919769,27.789513,50.642258,445.790924,-38.991756,82.616188,...,58.341286,2131.451660,-1270.859253,-47.280304,1117.687988,-469.864166,73.703926,588.587219,-159.614395,82.927773
2,roi1,667800,252600,3,262.668854,17.658516,26.511618,416.402771,-51.110100,51.083614,...,64.631248,1809.830200,-503.234863,18.854675,843.746643,-249.886932,54.688435,432.331055,-117.106590,44.694645
3,roi1,667900,252600,3,248.608994,1.748695,19.447910,399.852753,-56.740559,45.920238,...,81.518433,1844.627808,-428.580078,57.452545,821.924438,-237.290436,112.084305,427.219788,-125.443512,76.133369
4,roi1,668000,252600,3,258.873657,-7.505610,21.662420,426.105316,-74.219902,44.687603,...,61.442841,1752.874146,-645.925171,102.277191,916.252747,-267.664307,150.652328,484.190948,-123.407143,95.016434


In [6]:
# label annual composites

# creating name of features
feature_names = spectral_bands

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []

for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_composite, geotransform, epsg = geotiff_to_datacube(f'{folder_composites}annual_composite_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    
        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
    
        features[i,:] = dc_composite[irow,icol,:]

        
    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    

data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}annual_composite_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()
        

roi1
roi2
roi3


,roi,X,Y,land_cover,blue,green,red,nir,swir1,swir2
0,roi1,667600,252600,2,212.0,304.0,208.0,979.0,751.0,361.0
1,roi1,667700,252600,3,208.0,325.0,213.0,1392.0,850.0,465.0
2,roi1,667800,252600,3,205.0,215.0,196.0,970.0,312.0,153.0
3,roi1,667900,252600,3,203.0,321.0,193.0,1329.0,445.0,216.0
4,roi1,668000,252600,3,212.0,309.0,218.0,1224.0,588.0,312.0


In [9]:
# label annual composites

# creating name of features
seasons = ['spring','summer','autumn']
feature_names = [f'{band}_{season}' for band in spectral_bands for season in seasons]

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []


for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_spring, geotransform, epsg = geotiff_to_datacube(f'{folder_composites}seasonal_composite_spring_{roi}_{year}.tif')
    dc_summer, _, _ = geotiff_to_datacube(f'{folder_composites}seasonal_composite_summer_{roi}_{year}.tif')
    dc_autumn, _, _ = geotiff_to_datacube(f'{folder_composites}seasonal_composite_autumn_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    
        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
        
        for iband in range(len(spectral_bands)):
            features[i,iband*len(seasons)] = dc_spring[irow,icol,iband]
            features[i,iband*len(seasons)+1] = dc_summer[irow,icol,iband]
            features[i,iband*len(seasons)+2] = dc_autumn[irow,icol,iband]

    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    
data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}seasonal_composite_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()    

roi1
roi2
roi3


,roi,X,Y,land_cover,blue_spring,blue_summer,blue_autumn,green_spring,green_summer,green_autumn,...,red_autumn,nir_spring,nir_summer,nir_autumn,swir1_spring,swir1_summer,swir1_autumn,swir2_spring,swir2_summer,swir2_autumn
0,roi1,667600,252600,2,206.0,199.0,198.0,301.0,304.0,276.0,...,210.0,979.0,979.0,874.0,645.0,645.0,629.0,361.0,361.0,339.0
1,roi1,667700,252600,3,215.0,208.0,202.0,324.0,325.0,321.0,...,221.0,1223.0,1223.0,1243.0,850.0,850.0,850.0,473.0,469.0,467.0
2,roi1,667800,252600,3,187.0,182.0,182.0,218.0,215.0,202.0,...,127.0,970.0,970.0,934.0,311.0,310.0,304.0,153.0,153.0,149.0
3,roi1,667900,252600,3,183.0,203.0,173.0,322.0,321.0,252.0,...,165.0,1329.0,1329.0,1286.0,451.0,451.0,448.0,214.0,207.0,169.0
4,roi1,668000,252600,3,207.0,212.0,176.0,261.0,217.0,239.0,...,216.0,1224.0,1224.0,1198.0,588.0,588.0,585.0,317.0,317.0,280.0
